In [1]:
import gensim
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from util.dataset_util import *
from util.model_util import *
from keras.applications.vgg16 import VGG16
from keras.models import load_model

C:\Users\tianh\Desktop\environments\mlenv\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\tianh\Desktop\environments\mlenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
embedding_model = read_embedding_model('model/GoogleNews-vectors-negative300.bin')

In [3]:
model = None
load_saved = True
if load_saved:
    model = load_model('model/saved_model.h5')
else:
    model = create_model()

C:\Users\tianh\Desktop\environments\mlenv\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              1050624   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_4 (Dense)              (None, 300)               153900    
Total params: 18,542,188
Trainable params: 18,542,188
Non-trainable params: 0
_________________________________________________________________


In [5]:
for i in range(3):
    imgs, labels = read_cifar_batch('data/cifar10', 'data_batch_'+str(i%5+1), 'batches.meta', embedding_model, (224, 224), 1000)
    loss = model.fit(x=imgs, y=labels, epochs=10, verbose=1, validation_split=0.2)
    imgs, labels = read_cifar_batch('data/cifar100', 'train', 'meta', embedding_model, (224, 224), 1000)
    loss = model.fit(x=imgs, y=labels, epochs=10, verbose=1, validation_split=0.2)
    imgs, labels = read_caltech('data/caltech101', 10, (224, 224), embedding_model)
    loss = model.fit(x=imgs, y=labels, epochs=10, verbose=1, validation_split=0.2)

Train on 509 samples, validate on 128 samples
Epoch 1/10
509/509 [==============================] - 26s 51ms/step - loss: 0.1157 - val_loss: 0.1145
Epoch 2/10
509/509 [==============================] - 17s 34ms/step - loss: 0.1134 - val_loss: 0.1118
Epoch 3/10
509/509 [==============================] - 17s 34ms/step - loss: 0.1107 - val_loss: 0.1090
Epoch 4/10
509/509 [==============================] - 17s 34ms/step - loss: 0.1081 - val_loss: 0.1065
Epoch 5/10
509/509 [==============================] - 17s 34ms/step - loss: 0.1056 - val_loss: 0.1041
Epoch 6/10
509/509 [==============================] - 17s 34ms/step - loss: 0.1034 - val_loss: 0.1019
Epoch 7/10
509/509 [==============================] - 17s 34ms/step - loss: 0.1012 - val_loss: 0.0998
Epoch 8/10
509/509 [==============================] - 17s 34ms/step - loss: 0.0993 - val_loss: 0.0979
Epoch 9/10
509/509 [==============================] - 17s 34ms/step - loss: 0.0974 - val_loss: 0.0961
Epoch 10/10
509/509 [===============

KeyError: "word 'gerenuk' not in vocabulary"

In [6]:
predict('data/sample/truck1.jpg', model, embedding_model)

1/1 [==============================] - 1s 807ms/step


[('abating', 0.29838889837265015),
 ('Amazon', 0.27542173862457275),
 ('Chris_Halligan_PokerTek', 0.2752698063850403),
 ('viral_hemorrhagic_septicemia_VHS', 0.27129578590393066),
 ('Rocktober', 0.26739028096199036),
 ('necessarily', 0.26302453875541687),
 ('Health_Mary_Selecky', 0.26158663630485535),
 ('Fishburne', 0.26106157898902893),
 ('General_Willem_Wijnstekers', 0.26007768511772156),
 ('our', 0.25613293051719666)]

In [ ]:
model.save('model/saved_model.h5')